In [87]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
dirpath=os.chdir("C:/Users/P2190101/Desktop/NK Personal/NK Study/UNCC/6162 - KDD/KDD-Python-Assignments/MovieReviews/")
fileslist = os.listdir(dirpath)  # Get all the files in that directory
print("Files in %r: %s" % (dirpath, files))

Files in None: ['.ipynb_checkpoints', '16748.txt', '17108.txt', '17109.txt', '17110.txt', '17111.txt', '17116.txt', '17117.txt', '17118.txt', '17119.txt', '17139.txt', '17144.txt', '17145.txt', '17146.txt', '17147.txt', '17150.txt', '17185.txt', '17192.txt', '17219.txt', '17239.txt', '17243.txt', '17254.txt', '17255.txt', '17280.txt', '17300.txt', '17303.txt', '17341.txt', '17384.txt', '17391.txt', '17398.txt', '17399.txt', '17430.txt', '17431.txt', '17447.txt', '17457.txt', '17460.txt', '17501.txt', '17518.txt', '17532.txt', '17534.txt', '17578.txt', '17609.txt', '17610.txt', '17655.txt', '17662.txt', '17663.txt', '17695.txt', '17711.txt', '17713.txt', '17753.txt', '17757.txt', '17758.txt', '17761.txt', '17803.txt', '17811.txt', '17874.txt', '17879.txt', '17886.txt', '17896.txt', '17898.txt', '17902.txt', '17912.txt', '17933.txt', '17934.txt', '17945.txt', '17963.txt', '17971.txt', '17992.txt', '18004.txt', '18016.txt', '18032.txt', '18067.txt', '18068.txt', '18080.txt', '18087.txt', 

In [ ]:
import pandas as pd
import re
sub = 9999
df_list= []
#df = pd.DataFrame(columns=['Review','Writer'])
for file in fileslist:
    filecontent = open(file).read()
    #print(filecontent)
    flag= [int(i) for i in file.split('.') if i.isdigit()]
    #print(flag)
    for x in flag:
            if sub >= x:
                #print(x)
                df_list.append([filecontent, 0])
            if sub < x:
                 df_list.append([filecontent, 1])          
    
df_list


In [101]:
type(df_list)

list

In [106]:
len(df_list)

180

In [107]:
df = pd.DataFrame(df_list, columns =['Review','Writer']) 
df 

,Review,Writer
0,"DENNIS SCHWARTZ ""Movie Reviews and Poetry""\nUN...",1
1,"A brilliant, witty mock documentary of Jean Se...",1
2,NOSTALGHIA (director: Andrei Tarkovsky; cast: ...,1
3,PAYBACK (director: Brian Helgeland; cast:(Port...,1
4,WAKING NED DEVINE (director: Kirk Jones (III);...,1
...,...,...
175,"Well, either CALENDAR GIRL is a blatant attemp...",0
176,"For those who live in and around Philadelphia,...",0
177,"Starring: Suzy Amis, Bo Hopkins, Ian McKellen,...",0
178,"Starring: Jesse Bradford, Jeroen Krabbe, Lisa ...",0


In [108]:
df.Writer.unique()

array([1, 0], dtype=int64)

In [110]:
# This is to vectorize the text corpus. After these codes, the X object will be the input vector for machine learning models.
# When transform into vectors, we do NOT use the raw count of a word in a document. Instead, we use the word's tf-idf score in a document.
# max_df=0.5 means ingoring words that appear in more than 50% of the documents; min_df=2 means ignoring words that appear in less than 2 documents.

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True)

X = vectorizer.fit_transform(df.Review)

In [111]:
#X in the previuos step is hidimentional data, we need to use some dimentionality reduction technique. In this case, we will use SVD (Singular Value Decomposition), which is a common matrix decomposition technique.
#We want to reduce the dimentionality to 5.
#We have to re-normalize after we run our SVD on the dataset.
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

n_components = 3
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

In [112]:
X

array([[ 0.92676007,  0.37466333, -0.02726108],
       [ 0.86809119,  0.48277909,  0.11550772],
       [ 0.79731955,  0.51825264, -0.30934728],
       [ 0.99647531,  0.07015954,  0.04598465],
       [ 0.9297364 ,  0.19641039, -0.3114694 ],
       [ 0.96530604,  0.23691166, -0.1098049 ],
       [ 0.67303558,  0.72314318, -0.15520002],
       [ 0.59080971,  0.65173726,  0.4755864 ],
       [ 0.95300196,  0.25212777,  0.16798465],
       [ 0.84362981,  0.47156071, -0.25674742],
       [ 0.85510593,  0.39320137, -0.33791498],
       [ 0.83497086,  0.4744951 ,  0.27870785],
       [ 0.94619172,  0.24185741, -0.21500286],
       [ 0.91125048,  0.40258961, -0.08685718],
       [ 0.67805757,  0.43533742,  0.59221556],
       [ 0.99053568,  0.1362142 ,  0.01687489],
       [ 0.80046426,  0.49633596, -0.33601725],
       [ 0.97972942,  0.19382622, -0.05061289],
       [ 0.65865965,  0.55288434, -0.51037867],
       [ 0.97756043,  0.20796979,  0.0335287 ],
       [ 0.90968438,  0.39676421, -0.122

In [113]:
X.shape

(180, 3)

In [114]:
#Randomly select 80% (180*80% = 144) rows from X as the training set
import numpy as np
training_idx = np.random.choice(X.shape[0], size=144, replace=False)
X_training = X[training_idx, :]

In [115]:
#The remaining is the test set
test_idx = list(set(range(X.shape[0])) - set(training_idx))
X_test = X[test_idx, :]

In [117]:
labels = df.Writer

In [118]:
labels

0      1
1      1
2      1
3      1
4      1
      ..
175    0
176    0
177    0
178    0
179    0
Name: Writer, Length: 180, dtype: int64

In [119]:
#The same split for the labels list
labels_training = [labels[i] for i in training_idx]
labels_test = [labels[j] for j in test_idx]

In [120]:
#This is to use Naive Bayes Classifier to predict the label of each news article.
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_training, labels_training)

GaussianNB(priors=None, var_smoothing=1e-09)

In [121]:
#This is the prediction result.
gnb.predict(X_test)

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [122]:
#This is the confusion matrix.
import sklearn.metrics
sklearn.metrics.confusion_matrix(gnb.predict(X_test),labels_test)

array([[12,  1],
       [ 0, 23]], dtype=int64)

In [123]:
#This is the accuracy score.
sklearn.metrics.accuracy_score(gnb.predict(X_test),labels_test)

0.9722222222222222